# Оценка стоимости недвижимости с помощью нейронной сети


Чтобы запускать и редактировать код, сохраните копию этого ноутбука себе (File->Save a copy in Drive...). Свою копию вы сможете изменять и запускать.

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
!ls "/content/drive/My Drive/"

 BostonHousing.csv  'Colab Notebooks'   test.csv   train.csv


Загрузка данных с Google Drive

In [ ]:
import pandas as pd
data = pd.read_csv("/content/drive/My Drive/test.csv")

Отображение загруженных данных

In [ ]:
data

,Id,Open Date,City,City Group,Type,P1,P2,P3,P4,P5,P6,P7,P8,P9,P10,P11,P12,P13,P14,P15,P16,P17,P18,P19,P20,P21,P22,P23,P24,P25,P26,P27,P28,P29,P30,P31,P32,P33,P34,P35,P36,P37
0,0,01/22/2011,Niğde,Other,FC,1,4.0,4.0,4.0,1,2,5,4,5,5,5,3,4.0,0,0,0,2,0,5,5,3,1,4,0,0,0.0,0.0,2.0,3.0,0,0,0,0,0,0,0,0
1,1,03/18/2011,Konya,Other,IL,3,4.0,4.0,4.0,2,2,5,3,4,4,2,4,5.0,0,0,0,0,0,5,5,3,2,1,0,0,0.0,0.0,1.0,3.0,0,0,0,0,0,0,0,0
2,2,10/30/2013,Ankara,Big Cities,FC,3,4.0,4.0,4.0,2,2,5,4,4,5,4,5,5.0,0,0,0,0,0,5,5,5,5,5,0,0,0.0,0.0,2.0,3.0,0,0,0,0,0,0,0,0
3,3,05/06/2013,Kocaeli,Other,IL,2,4.0,4.0,4.0,2,3,5,4,5,4,3,4,5.0,0,0,0,0,4,4,4,3,2,2,0,0,0.0,0.0,2.0,3.0,0,4,0,0,0,0,0,0
4,4,07/31/2013,Afyonkarahisar,Other,FC,2,4.0,4.0,4.0,1,2,5,4,5,4,3,5,4.0,0,0,0,0,0,1,5,3,1,1,0,0,0.0,0.0,5.0,3.0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,99995,01/05/2000,Antalya,Other,FC,5,5.0,4.0,4.0,2,2,5,5,4,4,4,4,4.0,0,0,0,0,0,3,1,1,3,2,0,0,0.0,0.0,2.0,3.0,0,0,0,0,0,0,0,0
99996,99996,07/18/2011,Niğde,Other,IL,1,2.0,4.0,3.0,1,1,1,5,5,5,2,4,5.0,0,0,0,1,0,1,1,1,1,2,0,0,0.0,0.0,1.0,3.0,5,0,0,0,4,0,0,0
99997,99997,12/29/2012,İstanbul,Big Cities,IL,4,5.0,4.0,4.0,1,2,5,3,4,5,5,4,5.0,0,4,4,3,3,3,5,4,5,5,4,4,4.0,3.0,4.0,1.0,5,5,3,2,4,4,4,2
99998,99998,10/12/2013,İstanbul,Big Cities,FC,12,7.5,6.0,6.0,4,4,10,10,10,10,6,10,7.5,0,0,0,0,0,20,9,3,3,15,0,2,0.0,0.0,7.5,5.0,0,0,0,4,0,0,0,0


Удаление столбца ID из набора данных

In [ ]:
data = data.drop(columns=['Id'])

In [ ]:
data = data.drop(columns=['Open Date'])

In [ ]:
data

,City,City Group,Type,P1,P2,P3,P4,P5,P6,P7,P8,P9,P10,P11,P12,P13,P14,P15,P16,P17,P18,P19,P20,P21,P22,P23,P24,P25,P26,P27,P28,P29,P30,P31,P32,P33,P34,P35,P36,P37
0,Niğde,Other,FC,1,4.0,4.0,4.0,1,2,5,4,5,5,5,3,4.0,0,0,0,2,0,5,5,3,1,4,0,0,0.0,0.0,2.0,3.0,0,0,0,0,0,0,0,0
1,Konya,Other,IL,3,4.0,4.0,4.0,2,2,5,3,4,4,2,4,5.0,0,0,0,0,0,5,5,3,2,1,0,0,0.0,0.0,1.0,3.0,0,0,0,0,0,0,0,0
2,Ankara,Big Cities,FC,3,4.0,4.0,4.0,2,2,5,4,4,5,4,5,5.0,0,0,0,0,0,5,5,5,5,5,0,0,0.0,0.0,2.0,3.0,0,0,0,0,0,0,0,0
3,Kocaeli,Other,IL,2,4.0,4.0,4.0,2,3,5,4,5,4,3,4,5.0,0,0,0,0,4,4,4,3,2,2,0,0,0.0,0.0,2.0,3.0,0,4,0,0,0,0,0,0
4,Afyonkarahisar,Other,FC,2,4.0,4.0,4.0,1,2,5,4,5,4,3,5,4.0,0,0,0,0,0,1,5,3,1,1,0,0,0.0,0.0,5.0,3.0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,Antalya,Other,FC,5,5.0,4.0,4.0,2,2,5,5,4,4,4,4,4.0,0,0,0,0,0,3,1,1,3,2,0,0,0.0,0.0,2.0,3.0,0,0,0,0,0,0,0,0
99996,Niğde,Other,IL,1,2.0,4.0,3.0,1,1,1,5,5,5,2,4,5.0,0,0,0,1,0,1,1,1,1,2,0,0,0.0,0.0,1.0,3.0,5,0,0,0,4,0,0,0
99997,İstanbul,Big Cities,IL,4,5.0,4.0,4.0,1,2,5,3,4,5,5,4,5.0,0,4,4,3,3,3,5,4,5,5,4,4,4.0,3.0,4.0,1.0,5,5,3,2,4,4,4,2
99998,İstanbul,Big Cities,FC,12,7.5,6.0,6.0,4,4,10,10,10,10,6,10,7.5,0,0,0,0,0,20,9,3,3,15,0,2,0.0,0.0,7.5,5.0,0,0,0,4,0,0,0,0


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 40 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   City        100000 non-null  object 
 1   City Group  100000 non-null  object 
 2   Type        100000 non-null  object 
 3   P1          100000 non-null  int64  
 4   P2          100000 non-null  float64
 5   P3          100000 non-null  float64
 6   P4          100000 non-null  float64
 7   P5          100000 non-null  int64  
 8   P6          100000 non-null  int64  
 9   P7          100000 non-null  int64  
 10  P8          100000 non-null  int64  
 11  P9          100000 non-null  int64  
 12  P10         100000 non-null  int64  
 13  P11         100000 non-null  int64  
 14  P12         100000 non-null  int64  
 15  P13         100000 non-null  float64
 16  P14         100000 non-null  int64  
 17  P15         100000 non-null  int64  
 18  P16         100000 non-null  int64  
 19  P17

In [ ]:
data.dtypes

City           object
City Group     object
Type           object
P1              int64
P2            float64
P3            float64
P4            float64
P5              int64
P6              int64
P7              int64
P8              int64
P9              int64
P10             int64
P11             int64
P12             int64
P13           float64
P14             int64
P15             int64
P16             int64
P17             int64
P18             int64
P19             int64
P20             int64
P21             int64
P22             int64
P23             int64
P24             int64
P25             int64
P26           float64
P27           float64
P28           float64
P29           float64
P30             int64
P31             int64
P32             int64
P33             int64
P34             int64
P35             int64
P36             int64
P37             int64
dtype: object

In [ ]:
data_object = data.select_dtypes (include=[object])
data_object

,City,City Group,Type
0,Niğde,Other,FC
1,Konya,Other,IL
2,Ankara,Big Cities,FC
3,Kocaeli,Other,IL
4,Afyonkarahisar,Other,FC
...,...,...,...
99995,Antalya,Other,FC
99996,Niğde,Other,IL
99997,İstanbul,Big Cities,IL
99998,İstanbul,Big Cities,FC


In [ ]:
data_int = data.select_dtypes (include=['number'])
data_int

,P1,P2,P3,P4,P5,P6,P7,P8,P9,P10,P11,P12,P13,P14,P15,P16,P17,P18,P19,P20,P21,P22,P23,P24,P25,P26,P27,P28,P29,P30,P31,P32,P33,P34,P35,P36,P37
0,1,4.0,4.0,4.0,1,2,5,4,5,5,5,3,4.0,0,0,0,2,0,5,5,3,1,4,0,0,0.0,0.0,2.0,3.0,0,0,0,0,0,0,0,0
1,3,4.0,4.0,4.0,2,2,5,3,4,4,2,4,5.0,0,0,0,0,0,5,5,3,2,1,0,0,0.0,0.0,1.0,3.0,0,0,0,0,0,0,0,0
2,3,4.0,4.0,4.0,2,2,5,4,4,5,4,5,5.0,0,0,0,0,0,5,5,5,5,5,0,0,0.0,0.0,2.0,3.0,0,0,0,0,0,0,0,0
3,2,4.0,4.0,4.0,2,3,5,4,5,4,3,4,5.0,0,0,0,0,4,4,4,3,2,2,0,0,0.0,0.0,2.0,3.0,0,4,0,0,0,0,0,0
4,2,4.0,4.0,4.0,1,2,5,4,5,4,3,5,4.0,0,0,0,0,0,1,5,3,1,1,0,0,0.0,0.0,5.0,3.0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,5,5.0,4.0,4.0,2,2,5,5,4,4,4,4,4.0,0,0,0,0,0,3,1,1,3,2,0,0,0.0,0.0,2.0,3.0,0,0,0,0,0,0,0,0
99996,1,2.0,4.0,3.0,1,1,1,5,5,5,2,4,5.0,0,0,0,1,0,1,1,1,1,2,0,0,0.0,0.0,1.0,3.0,5,0,0,0,4,0,0,0
99997,4,5.0,4.0,4.0,1,2,5,3,4,5,5,4,5.0,0,4,4,3,3,3,5,4,5,5,4,4,4.0,3.0,4.0,1.0,5,5,3,2,4,4,4,2
99998,12,7.5,6.0,6.0,4,4,10,10,10,10,6,10,7.5,0,0,0,0,0,20,9,3,3,15,0,2,0.0,0.0,7.5,5.0,0,0,0,4,0,0,0,0


In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
subdata_encoded = data_object.apply(le.fit_transform)

In [ ]:
subdata_encoded

,City,City Group,Type
0,38,1,1
1,27,1,2
2,3,0,1
3,26,1,2
4,1,1,1
...,...,...,...
99995,4,1,1
99996,38,1,2
99997,54,0,2
99998,54,0,1


In [ ]:
data_2 = pd.concat([subdata_encoded, data_int], axis=1)

In [ ]:
data_2

,City,City Group,Type,P1,P2,P3,P4,P5,P6,P7,P8,P9,P10,P11,P12,P13,P14,P15,P16,P17,P18,P19,P20,P21,P22,P23,P24,P25,P26,P27,P28,P29,P30,P31,P32,P33,P34,P35,P36,P37
0,38,1,1,1,4.0,4.0,4.0,1,2,5,4,5,5,5,3,4.0,0,0,0,2,0,5,5,3,1,4,0,0,0.0,0.0,2.0,3.0,0,0,0,0,0,0,0,0
1,27,1,2,3,4.0,4.0,4.0,2,2,5,3,4,4,2,4,5.0,0,0,0,0,0,5,5,3,2,1,0,0,0.0,0.0,1.0,3.0,0,0,0,0,0,0,0,0
2,3,0,1,3,4.0,4.0,4.0,2,2,5,4,4,5,4,5,5.0,0,0,0,0,0,5,5,5,5,5,0,0,0.0,0.0,2.0,3.0,0,0,0,0,0,0,0,0
3,26,1,2,2,4.0,4.0,4.0,2,3,5,4,5,4,3,4,5.0,0,0,0,0,4,4,4,3,2,2,0,0,0.0,0.0,2.0,3.0,0,4,0,0,0,0,0,0
4,1,1,1,2,4.0,4.0,4.0,1,2,5,4,5,4,3,5,4.0,0,0,0,0,0,1,5,3,1,1,0,0,0.0,0.0,5.0,3.0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,4,1,1,5,5.0,4.0,4.0,2,2,5,5,4,4,4,4,4.0,0,0,0,0,0,3,1,1,3,2,0,0,0.0,0.0,2.0,3.0,0,0,0,0,0,0,0,0
99996,38,1,2,1,2.0,4.0,3.0,1,1,1,5,5,5,2,4,5.0,0,0,0,1,0,1,1,1,1,2,0,0,0.0,0.0,1.0,3.0,5,0,0,0,4,0,0,0
99997,54,0,2,4,5.0,4.0,4.0,1,2,5,3,4,5,5,4,5.0,0,4,4,3,3,3,5,4,5,5,4,4,4.0,3.0,4.0,1.0,5,5,3,2,4,4,4,2
99998,54,0,1,12,7.5,6.0,6.0,4,4,10,10,10,10,6,10,7.5,0,0,0,0,0,20,9,3,3,15,0,2,0.0,0.0,7.5,5.0,0,0,0,4,0,0,0,0


In [ ]:
#Среднее значение
mymean = data_2.mean(axis=0)
#Стандартное отклонение
mystd = data_2.std(axis=0)
data_2 -= mymean
data_2 /= mystd
data_2

,City,City Group,Type,P1,P2,P3,P4,P5,P6,P7,P8,P9,P10,P11,P12,P13,P14,P15,P16,P17,P18,P19,P20,P21,P22,P23,P24,P25,P26,P27,P28,P29,P30,P31,P32,P33,P34,P35,P36,P37
0,0.209831,0.985540,-0.722552,-1.097786,-0.299598,-0.255682,-0.382327,-0.928919,-0.575867,-0.137938,-0.541755,-0.147642,-0.260246,0.816336,-1.167701,-1.109555,-0.512696,-0.602474,-0.550777,0.346586,-0.563753,-0.033034,0.118795,0.181161,-1.049293,0.076289,-0.588546,-0.596772,-0.554054,-0.534271,-0.577427,-0.047087,-0.483423,-0.517053,-0.48917,-0.643357,-0.450050,-0.567656,-0.517343,-0.580408
1,-0.312024,0.985540,1.140512,-0.386791,-0.299598,-0.255682,-0.382327,0.009772,-0.575867,-0.137938,-1.123661,-0.734968,-0.827268,-0.634258,-0.601252,-0.088458,-0.512696,-0.602474,-0.550777,-0.475564,-0.563753,-0.033034,0.118795,0.181161,-0.315717,-0.559417,-0.588546,-0.596772,-0.554054,-0.534271,-1.045440,-0.047087,-0.483423,-0.517053,-0.48917,-0.643357,-0.450050,-0.567656,-0.517343,-0.580408
2,-1.450616,-1.014662,-0.722552,-0.386791,-0.299598,-0.255682,-0.382327,0.009772,-0.575867,-0.137938,-0.541755,-0.734968,-0.260246,0.332805,-0.034803,-0.088458,-0.512696,-0.602474,-0.550777,-0.475564,-0.563753,-0.033034,0.118795,0.972912,1.885013,0.288191,-0.588546,-0.596772,-0.554054,-0.534271,-0.577427,-0.047087,-0.483423,-0.517053,-0.48917,-0.643357,-0.450050,-0.567656,-0.517343,-0.580408
3,-0.359465,0.985540,1.140512,-0.742289,-0.299598,-0.255682,-0.382327,0.009772,0.077117,-0.137938,-0.541755,-0.147642,-0.827268,-0.150726,-0.601252,-0.088458,-0.512696,-0.602474,-0.550777,-0.475564,0.756149,-0.205879,-0.158027,0.181161,-0.315717,-0.347515,-0.588546,-0.596772,-0.554054,-0.534271,-0.577427,-0.047087,-0.483423,1.216091,-0.48917,-0.643357,-0.450050,-0.567656,-0.517343,-0.580408
4,-1.545499,0.985540,-0.722552,-0.742289,-0.299598,-0.255682,-0.382327,-0.928919,-0.575867,-0.137938,-0.541755,-0.147642,-0.827268,-0.150726,-0.034803,-1.109555,-0.512696,-0.602474,-0.550777,-0.475564,-0.563753,-0.724414,0.118795,0.181161,-1.049293,-0.559417,-0.588546,-0.596772,-0.554054,-0.534271,0.826611,-0.047087,-0.483423,-0.517053,-0.48917,-0.643357,-0.450050,-0.567656,-0.517343,-0.580408
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,-1.403175,0.985540,-0.722552,0.324203,0.400258,-0.255682,-0.382327,0.009772,-0.575867,-0.137938,0.040152,-0.734968,-0.827268,0.332805,-0.601252,-1.109555,-0.512696,-0.602474,-0.550777,-0.475564,-0.563753,-0.378724,-0.988493,-0.610591,0.417860,-0.347515,-0.588546,-0.596772,-0.554054,-0.534271,-0.577427,-0.047087,-0.483423,-0.517053,-0.48917,-0.643357,-0.450050,-0.567656,-0.517343,-0.580408
99996,0.209831,0.985540,1.140512,-1.097786,-1.699310,-0.255682,-1.347740,-0.928919,-1.228852,-1.973997,0.040152,-0.147642,-0.260246,-0.634258,-0.601252,-0.088458,-0.512696,-0.602474,-0.550777,-0.064489,-0.563753,-0.724414,-0.988493,-0.610591,-1.049293,-0.347515,-0.588546,-0.596772,-0.554054,-0.534271,-1.045440,-0.047087,0.676810,-0.517053,-0.48917,-0.643357,0.403663,-0.567656,-0.517343,-0.580408
99997,0.968892,-1.014662,1.140512,-0.031294,0.400258,-0.255682,-0.382327,-0.928919,-0.575867,-0.137938,-1.123661,-0.734968,-0.260246,0.816336,-0.601252,-0.088458,-0.512696,1.243439,0.710253,0.757661,0.426173,-0.378724,0.118795,0.577036,1.885013,0.288191,1.318525,1.321940,1.177818,0.842195,0.358598,-1.168209,0.676810,1.649377,0.26625,0.659737,0.403663,0.671206,0.533692,0.612007
99998,0.968892,-1.014662,-0.722552,2.812682,2.149898,2.119162,1.548497,1.887153,0.730102,2.157134,2.949682,2.788987,2.574863,1.299868,2.797444,2.464285,-0.512696,-0.602474,-0.550777,-0.475564,-0.563753,2.559641,1.226083,0.181161,0.417860,2.407212,-0.588546,0.362584,-0.554054,-0.534271,1.996643,1.074035,-0.483423,-0.517053,-0.48917,1.962832,-0.450050,-0.567656,-0.517343,-0.580408


In [ ]:
ydata = data.iloc[:, 39]

In [ ]:
ydata

0        0
1        0
2        0
3        0
4        0
        ..
99995    0
99996    0
99997    2
99998    0
99999    0
Name: P37, Length: 100000, dtype: int64

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data_2, ydata, test_size=0.33, random_state=42)

In [ ]:
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense

In [ ]:
model = Sequential()
model.add(Dense(32, activation='relu', input_shape=(X_train.shape[1],)))
model.add(Dense(128, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(1))

In [ ]:
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 32)                1312      
_________________________________________________________________
dense_1 (Dense)              (None, 128)               4224      
_________________________________________________________________
dense_2 (Dense)              (None, 512)               66048     
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 513       
Total params: 72,097
Trainable params: 72,097
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
model.compile(optimizer='adam', loss='mse', metrics=['mse'])

In [ ]:
model.fit(X_train, y_train, epochs=20, batch_size =32, validation_split= 0.2, verbose=2)

Epoch 1/20
1675/1675 - 5s - loss: 0.1401 - mse: 0.1401 - val_loss: 0.0154 - val_mse: 0.0154
Epoch 2/20
1675/1675 - 4s - loss: 0.0090 - mse: 0.0090 - val_loss: 0.0067 - val_mse: 0.0067
Epoch 3/20
1675/1675 - 4s - loss: 0.0036 - mse: 0.0036 - val_loss: 0.0022 - val_mse: 0.0022
Epoch 4/20
1675/1675 - 4s - loss: 0.0018 - mse: 0.0018 - val_loss: 0.0018 - val_mse: 0.0018
Epoch 5/20
1675/1675 - 4s - loss: 0.0016 - mse: 0.0016 - val_loss: 0.0018 - val_mse: 0.0018
Epoch 6/20
1675/1675 - 3s - loss: 0.0011 - mse: 0.0011 - val_loss: 4.7142e-04 - val_mse: 4.7142e-04
Epoch 7/20
1675/1675 - 4s - loss: 5.6251e-04 - mse: 5.6251e-04 - val_loss: 4.7058e-04 - val_mse: 4.7058e-04
Epoch 8/20
1675/1675 - 4s - loss: 5.7342e-04 - mse: 5.7342e-04 - val_loss: 5.5605e-04 - val_mse: 5.5605e-04
Epoch 9/20
1675/1675 - 4s - loss: 4.5055e-04 - mse: 4.5055e-04 - val_loss: 2.3533e-04 - val_mse: 2.3533e-04
Epoch 10/20
1675/1675 - 4s - loss: 7.0357e-04 - mse: 7.0357e-04 - val_loss: 8.1998e-05 - val_mse: 8.1998e-05
Epoch 1

In [ ]:
pred = model.predict(X_test) 

In [ ]:
print(pred)

[[ 1.0044804e+00]
 [-1.4473498e-04]
 [-2.9778481e-04]
 ...
 [-1.2232363e-04]
 [-1.5708059e-04]
 [ 2.0176919e+00]]


In [ ]:
y_test = y_test.reset_index(drop=True)

In [ ]:
print("Предсказанная стоимость:", pred[12][0], ", правильная стоимость:", y_test[12])

Предсказанная стоимость: 448.19504 , правильная стоимость: 3452382.0


In [ ]:
vec = np.array([])
for i in np.arange(0, len(pred)):
  vec = np.append(vec, pred[i][0])
vec  

array([ 756.87860107,  184.28530884,  547.85455322,  257.45007324,
        824.98840332,  734.53369141, 1381.21582031, 1241.80505371,
        690.22436523,  647.8302002 ,  222.54443359,  491.52151489,
        448.19503784,  564.63104248,  177.95565796,  680.05444336,
        591.68579102,  604.00213623,  689.32824707,  567.23931885,
       1207.3470459 ,  209.73768616,  695.67199707,  500.04519653,
        403.44717407,  525.94287109,  791.93579102,  241.94688416,
        748.2154541 ,  610.39453125,  587.58978271,  226.48893738,
        237.6164856 ,  540.11895752,  736.15905762,  278.31033325,
        394.52676392,  332.35321045,  909.68109131,  273.62954712,
        322.50704956,  299.21658325,  995.67712402, 1240.71984863,
        261.82040405,  244.54437256])

In [ ]:
diff = np.array([])

for i in np.arange(0, len(y_test)):
    diff = np.append(diff, np.abs(pred[i][0] - y_test[i]))

In [ ]:
np.sort(diff)[-10:-1]

array([5337284.05311584, 5434684.31420898, 5653420.64678955,
       5786904.67175293, 6836160.49295044, 7494444.1697998 ,
       8903784.78341675, 9262063.77563477, 9651824.05712891])

In [ ]:
vec

array([ 756.87860107,  184.28530884,  547.85455322,  257.45007324,
        824.98840332,  734.53369141, 1381.21582031, 1241.80505371,
        690.22436523,  647.8302002 ,  222.54443359,  491.52151489,
        448.19503784,  564.63104248,  177.95565796,  680.05444336,
        591.68579102,  604.00213623,  689.32824707,  567.23931885,
       1207.3470459 ,  209.73768616,  695.67199707,  500.04519653,
        403.44717407,  525.94287109,  791.93579102,  241.94688416,
        748.2154541 ,  610.39453125,  587.58978271,  226.48893738,
        237.6164856 ,  540.11895752,  736.15905762,  278.31033325,
        394.52676392,  332.35321045,  909.68109131,  273.62954712,
        322.50704956,  299.21658325,  995.67712402, 1240.71984863,
        261.82040405,  244.54437256])

In [ ]:
sorted = np.sort(pred, )
sorted

array([[ 756.8786 ],
       [ 184.28531],
       [ 547.85455],
       [ 257.45007],
       [ 824.9884 ],
       [ 734.5337 ],
       [1381.2158 ],
       [1241.805  ],
       [ 690.22437],
       [ 647.8302 ],
       [ 222.54443],
       [ 491.5215 ],
       [ 448.19504],
       [ 564.63104],
       [ 177.95566],
       [ 680.05444],
       [ 591.6858 ],
       [ 604.00214],
       [ 689.32825],
       [ 567.2393 ],
       [1207.347  ],
       [ 209.73769],
       [ 695.672  ],
       [ 500.0452 ],
       [ 403.44717],
       [ 525.9429 ],
       [ 791.9358 ],
       [ 241.94688],
       [ 748.21545],
       [ 610.39453],
       [ 587.5898 ],
       [ 226.48894],
       [ 237.61649],
       [ 540.11896],
       [ 736.15906],
       [ 278.31033],
       [ 394.52676],
       [ 332.3532 ],
       [ 909.6811 ],
       [ 273.62955],
       [ 322.50705],
       [ 299.21658],
       [ 995.6771 ],
       [1240.7198 ],
       [ 261.8204 ],
       [ 244.54437]], dtype=float32)

In [ ]:
for i in np.arange(0, len(y_test)):
    print("Предсказанная стоимость:", pred[i][0], ", правильная стоимость:", y_test[i], 'разница: ', np.abs(pred[i][0] - y_test[i]))

Предсказанная стоимость: 756.8786 , правильная стоимость: 2792031.0 разница:  2791274.121398926
Предсказанная стоимость: 184.28531 , правильная стоимость: 4467728.0 разница:  4467543.714691162
Предсказанная стоимость: 547.85455 , правильная стоимость: 1734634.0 разница:  1734086.1454467773
Предсказанная стоимость: 257.45007 , правильная стоимость: 2267425.0 разница:  2267167.549926758
Предсказанная стоимость: 824.9884 , правильная стоимость: 4263629.0 разница:  4262804.01159668
Предсказанная стоимость: 734.5337 , правильная стоимость: 1882131.0 разница:  1881396.4663085938
Предсказанная стоимость: 1381.2158 , правильная стоимость: 2551252.0 разница:  2549870.7841796875
Предсказанная стоимость: 1241.805 , правильная стоимость: 4882985.0 разница:  4881743.194946289
Предсказанная стоимость: 690.22437 , правильная стоимость: 9262754.0 разница:  9262063.775634766
Предсказанная стоимость: 647.8302 , правильная стоимость: 7495092.0 разница:  7494444.169799805
Предсказанная стоимость: 222.5444

In [ ]:
СС_tuner = np.corrcoef(vec, y_test)
СС_tuner = СС_tuner[0][1]
print(f'Коэффициаент корреляция с истинными данными: {СС_tuner}')

Коэффициаент корреляция с истинными данными: 0.03919857478124118
